In [ ]:
%load_ext lab_black

Step: Load data

In [2]:
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit


Add that teams have lines 

In [12]:
import itertools
import random
from datetime import datetime, timedelta

def generate_dates(start_date, end_date, day_of_week):
    dates = []
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    total_days = (end_date - start_date).days + 1
    for day_number in range(total_days):
        current_day = start_date + timedelta(days=day_number)
        if current_day.weekday() == day_of_week:
            dates.append(current_day)
    return dates

def schedule_matches(leagues, court_details, start_date, end_date, days_of_week, times, num_matches, preferred_court=None):
    schedules = {league: list(itertools.combinations(teams, 2)) for league, teams in leagues.items()}
    for schedule in schedules.values():
        random.shuffle(schedule)

    parsed_times = [datetime.strptime(time, "%I:%M %p") for time in times]
    
    dates = []
    for day_of_week in days_of_week:
        dates.extend(generate_dates(start_date, end_date, day_of_week))
    
    court_slots = [(location, court_number) for location, court_numbers in court_details.items() for court_number in court_numbers]
    date_time_slots = [datetime.combine(date, time.time()) for date in dates for time in parsed_times]
    
    all_slots = list(itertools.product(date_time_slots, court_slots))
    random.shuffle(all_slots)

    matches = {league: [] for league in leagues}
    occupied_courts = {}
    for league, schedule in schedules.items():
        for i in range(len(schedule)):
            if len(matches[league]) >= num_matches:
                break
            for slot in all_slots:
                date_time, (court_location, court_number) = slot
                court_key = (court_location, court_number)
                if court_key not in occupied_courts or occupied_courts[court_key] <= date_time - timedelta(hours=1.5):
                    if preferred_court and court_location != preferred_court:
                        continue
                    match = {}
                    match['team1'], match['team2'] = schedule[i]
                    match['date_time'] = date_time
                    match['court_location'] = court_location
                    match['court_number'] = court_number
                    occupied_courts[court_key] = date_time
                    matches[league].append(match)
                    all_slots.remove(slot)
                    break
    
    for matches_in_league in matches.values():
        matches_in_league.sort(key=lambda x: x['date_time'])
    
    return matches

leagues = {
    "Men's 4.0 Singles": ["Team A", "Team B", "Team C", "Team D"],
    "Women's 4.0 Singles": ["Team E", "Team F", "Team G", "Team H"]
}
court_details = {
    'Elmira': [1,2,3,4],
    'Southern boundaries': [3,4,5],
    'Whipoorwill': [6,7,8],
    'Garrett': [1,2,3,4],
    'Rock Quarry': [1,2,4]
}
start_date = "2023-08-01"
end_date = "2023-09-30"
days_of_week = [1, 3] # Tuesday and Thursday
times = ["6:30 PM", "7:30 PM", "8:30 PM"]
num_matches = 6
preferred_court = "Elmira"

matches = schedule_matches(leagues, court_details, start_date, end_date, days_of_week, times, num_matches, preferred_court)
for league, matches_in_league in matches.items():
    print(f"{league} Matches:")
    for match in matches_in_league:
        print(f"{match['team1']} vs {match['team2']} at {match['court_location']} Court {match['court_number']} on {match['date_time'].strftime('%Y-%m-%d %I:%M %p')}")


Men's 4.0 Singles Matches:
Team B vs Team C at Elmira Court 4 on 2023-08-17 07:30 PM
Team B vs Team D at Elmira Court 3 on 2023-08-17 08:30 PM
Team C vs Team D at Elmira Court 3 on 2023-09-05 07:30 PM
Team A vs Team D at Elmira Court 2 on 2023-09-05 08:30 PM
Team A vs Team C at Elmira Court 1 on 2023-09-14 07:30 PM
Team A vs Team B at Elmira Court 4 on 2023-09-26 08:30 PM
Women's 4.0 Singles Matches:
Team E vs Team H at Elmira Court 2 on 2023-09-14 06:30 PM
Team F vs Team G at Elmira Court 1 on 2023-09-19 08:30 PM
Team E vs Team G at Elmira Court 3 on 2023-09-26 06:30 PM
Team E vs Team F at Elmira Court 3 on 2023-09-26 08:30 PM
Team G vs Team H at Elmira Court 4 on 2023-09-28 06:30 PM
Team F vs Team H at Elmira Court 3 on 2023-09-28 06:30 PM
